# Exercise Sheet 1: Python Basics

This first  exercise sheet tests the basic functionalities of the Python programming language in the context of a simple prediction task. We consider the problem of predicting health risk of subjects from personal data and habits. We first use for this task a decision tree

![](tree.png)

adapted from the webpage http://www.refactorthis.net/post/2013/04/10/Machine-Learning-tutorial-How-to-create-a-decision-tree-in-RapidMiner-using-the-Titanic-passenger-data-set.aspx. For this exercise sheet, you are required to use only pure Python, and to not import any module, including numpy. In exercise sheet 2, the nearest neighbor part of this exercise sheet will be revisited with numpy.

## Classifying a single instance (15 P)

* Create a function that takes as input a tuple containing values for attributes (smoker,age,diet), and computes the output of the decision tree.
* Test your function on the tuple `('yes',31,'good')`,

In [10]:
def classify(x):
    dec = ''
    if (x[0] == 'yes' and x[1] > 29.5) or (x[0] == 'no' and x[2] == 'poor'):
        dec = 'more'
    else:
        dec = 'less'
    
    return dec

classify(('yes',31,'good'))

'more'

## Reading a dataset from a text file (10 P)

The file `health-test.txt` contains several fictious records of personal data and habits.

* Read the file automatically using the methods introduced during the lecture.
* Represent the dataset as a list of tuples.

In [11]:
Data = []
for line in open('health-test.txt','r'):
    temp = str.split(line,',')
    Data.append( ( temp[0],float(temp[1]),temp[2].rstrip('\n') ) )
Data

[('yes', 21.0, 'poor'),
 ('no', 50.0, 'good'),
 ('no', 23.0, 'good'),
 ('yes', 45.0, 'poor'),
 ('yes', 51.0, 'good'),
 ('no', 60.0, 'good'),
 ('no', 15.0, 'poor'),
 ('no', 18.0, 'good')]

## Applying the decision tree to the dataset (15 P)

* Apply the decision tree to all points in the dataset, and compute the percentage of them that are classified as "more risk".

In [12]:
n = len(Data)
r = sum([classify(x)=='more' for x in Data])
percentage = float(r)/float(n)*100
print percentage,"%"

37.5 %


## Learning from examples (10 P)

Suppose that instead of relying on a fixed decision tree, we would like to use a data-driven approach where data points are classified based on a set of training observations manually labeled by experts. Such labeled dataset is available in the file `health-train.txt`. The first three columns have the same meaning than for `health-test.txt`, and the last column corresponds to the labels.

* Write a procedure that reads this file and converts it into a list of pairs. The first element of each pair is a triplet of attributes, and the second element is the label.

In [13]:
Dtrain = []
for line in open('health-train.txt','r'):
    temp = str.split(line,',')
    Dtrain.append( (( temp[0],float(temp[1]),temp[2]),temp[3].rstrip('\n') ) )

Dtrain

[(('yes', 54.0, 'good'), 'less'),
 (('no', 55.0, 'good'), 'less'),
 (('no', 26.0, 'good'), 'less'),
 (('yes', 40.0, 'good'), 'more'),
 (('yes', 25.0, 'poor'), 'less'),
 (('no', 13.0, 'poor'), 'more'),
 (('no', 15.0, 'good'), 'less'),
 (('no', 50.0, 'poor'), 'more'),
 (('yes', 33.0, 'good'), 'more'),
 (('no', 35.0, 'good'), 'less'),
 (('no', 41.0, 'good'), 'less'),
 (('yes', 30.0, 'poor'), 'more'),
 (('no', 39.0, 'poor'), 'more'),
 (('no', 20.0, 'good'), 'less'),
 (('yes', 18.0, 'poor'), 'less'),
 (('yes', 55.0, 'good'), 'more')]

## Nearest neighbor classifier (25 P)

We consider the nearest neighbor algorithm that classifies test points following the label of the nearest neighbor in the training data. For this, we need to define a distance function between data points. We define it to be

`d(a,b) = (a[0]!=b[0])+((a[1]-b[1])/50.0)**2+(a[2]!=b[2])`

where `a` and `b` are two tuples corrsponding to the attributes of two data points.

* Write a function that retrieves for a test point the nearest neighbor in the training set, and classifies the test point accordingly.
* Test your function on the tuple `('yes',31,'good')`

In [14]:
import sys
def nnclassify(x):
    ndistance = sys.maxsize
    nn = ''  
    for i in Dtrain:
        train = i[0]
        d = (x[0]!=train[0])+((x[1]-train[1])/50.0)**2+(x[2]!=train[2])
        if(ndistance > d):
            ndistance = d
            nn = i
    
    return nn,ndistance

    
a,d = nnclassify(('yes',31,'good'))
print 'The nearest Neighbour is',a[0] ,'with distance',d
print 'The prediction is ',a[1],' risk'

The nearest Neighbour is ('yes', 33.0, 'good') with distance 0.0016
The prediction is  more  risk


* Apply both the decision tree and nearest neighbor classifiers on the test set, and find the data point(s) for which the two classifiers disagree, and with which probability it happens.

In [15]:
Different = []
n = len(Data)
c = 0
for i in Data:
    first = classify(i)
    nn,nd= nnclassify(i)
    second = nn[1]
    if(first != second):
        Different.append(i)
        c+=1
print 'Data points that disagree ',Different
print 'Probability ', float(c)/float(n)

Data points that disagree  [('yes', 51.0, 'good')]
Probability  0.125


One problem of simple nearest neighbors is that one needs to compare the point to predict to all data points in the training set. This can be slow for datasets of thousands of points or more. Alternatively, some classifiers train a model first, and then use it to classify the data.

## Nearest mean classifier (25 P)

We consider one such trainable model, which operates in two steps:

(1) Compute the average point for each class, (2) classify new points to be of the class whose average point is nearest to the point to predict.

For this classifier, we convert the attributes smoker and diet to real values (for smoker: yes=1.0 and no=0.0, and for diet: good=0.0 and poor=1.0), and use the modified distance function:

`d(a,b) = (a[0]-b[0])**2+((a[1]-b[1])/50.0)**2+(a[2]-b[2])**2`

We adopt an object-oriented approach for building this classifier.

* Implement the methods `train` and `predict` of the class `NearestMeanClassifier`.

In [16]:
class NearestMeanClassifier:
    
    # Training method that takes as input a dataset
    # and produces two internal vectors corresponding
    # to the mean of each class.        
    def train(self,dataset):
        lc,mc = 0.0,0.0
        self.lclass=[0.0,0.0,0.0]
        self.mclass=[0.0,0.0,0.0]
        for x in dataset:
            p = list(x[0])
            p[0] = (p[0] == 'yes')
            p[2] = (p[2] == 'poor')
            
            if(x[1] == 'less'):
                lc+=1
                self.lclass = [self.lclass[i]+p[i] for i in range(len(p))]
                
            else:
                mc+=1
                self.mclass = [self.mclass[i]+p[i] for i in range(len(p))]
        
        if lc != 0:
            self.lclass = [x / lc for x in self.lclass]
        if mc != 0:
            self.mclass = [x / mc for x in self.mclass]

    
    # Prediction method that takes as input a new data
    # point and predicts it to belong to the class with
    # nearest mean.
    def predict(self,x):
        D1 = ( (x[0]=='yes')-self.lclass[0])**2+((x[1]-self.lclass[1])/50.0)**2
        +( (x[2]=='poor')-self.lclass[2])**2
        D2 = ( (x[0]=='yes')-self.mclass[0])**2+((x[1]-self.mclass[1])/50.0)**2
        +( (x[2]=='poor')-self.mclass[2])**2
        if D1<D2 :
            return 'less'
        else:
            return 'more'

* Build an object of class `NearestMeanClassifier`, train it on the training data, and print the mean vector for each class.

In [17]:
a = NearestMeanClassifier()
a.train(Dtrain)
print 'the less risk class vector ',a.lclass
print 'the more risk class vector ',a.mclass

the less risk class vector  [0.3333333333333333, 32.111111111111114, 0.2222222222222222]
the more risk class vector  [0.5714285714285714, 37.142857142857146, 0.5714285714285714]


* Predict the test data using the nearest mean classifier and print all test examples for which all three classifiers (decision tree, nearest neighbor and nearest mean) agree.

In [18]:
t1 = [a.predict(x) for x in Data]
t2 = [classify(x) for x in Data]
t3 = [nnclassify(x)[0][1] for x in Data]
answer = []
i = 0
for x in Data:
    if t1[i]==t2[i] == t3[i]:
        answer.append(x)
    i+=1

    #had to print it this way as the PDF truncate some of it
print 'Data agreed by the 3 classifiers : \n'
for i in answer: 
    print i,'\n'


Data agreed by the 3 classifiers : 

('no', 50.0, 'good') 

('no', 23.0, 'good') 

('yes', 45.0, 'poor') 

('no', 60.0, 'good') 

('no', 18.0, 'good') 

